# 📝 Exercise M1.05

The goal of this exercise is to evaluate the impact of feature preprocessing
on a pipeline that uses a decision-tree-based classifier instead of a logistic
regression.

- The first question is to empirically evaluate whether scaling numerical
  features is helpful or not;
- The second question is to evaluate whether it is empirically better (both
  from a computational and a statistical perspective) to use integer coded or
  one-hot encoded categories.

In [1]:
import pandas as pd

adult_census = pd.read_csv("../datasets/adult-census.csv")

In [2]:
target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

As in the previous notebooks, we use the utility `make_column_selector` to
select only columns with a specific data type. Besides, we list in advance all
categories for the categorical columns.

In [4]:
from sklearn.compose import make_column_selector as selector

# define selector
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

# define column names for each dtype
numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

In [5]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# define transformers
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

## Reference pipeline (no numerical scaling and integer-coded categories)

First let's time the pipeline we used in the main notebook to serve as a
reference:

In [7]:
import time

from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import HistGradientBoostingClassifier

categorical_preprocessor = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1
)
preprocessor = ColumnTransformer(
    [("categorical", categorical_preprocessor, categorical_columns)],
    remainder="passthrough",
)

model = make_pipeline(preprocessor, HistGradientBoostingClassifier())

start = time.time()
cv_results = cross_validate(model, data, target)
elapsed_time = time.time() - start

scores = cv_results["test_score"]

print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} ± {scores.std():.3f} "
    f"with a fitting time of {elapsed_time:.3f}"
)

The mean cross-validation accuracy is: 0.873 ± 0.002 with a fitting time of 2.841


## Scaling numerical features

Let's write a similar pipeline that also scales the numerical features using
`StandardScaler` (or similar):

In [10]:
# Write your code here.
import time
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

# define selector
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

# define column names for each dtype
numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

# define transformers
categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
numerical_preprocessor = StandardScaler()

# define column transformer
preprocessor = ColumnTransformer(
    [
        ("ordinal-encoder", categorical_preprocessor, categorical_columns),
        ("standard-scaler", numerical_preprocessor, numerical_columns)
    ]
)

# make pipeline
model = make_pipeline(preprocessor, HistGradientBoostingClassifier())

# running cv on entire dataset to estimate generalisability
start_time = time.time()
cv_results = cross_validate(model, data, target, cv=5)
elapsed_time = time.time() - start_time
scores = cv_results["test_score"]

print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} ± {scores.std():.3f} "
    f"with a fitting time of {elapsed_time:.3f}"
)


The mean cross-validation accuracy is: 0.873 ± 0.002 with a fitting time of 2.902


There was no improvement in model performance, and a slight worsening of computation performance.

## One-hot encoding of categorical variables

We observed that integer coding of categorical variables can be very
detrimental for linear models. However, it does not seem to be the case for
`HistGradientBoostingClassifier` models, as the cross-validation score of the
reference pipeline with `OrdinalEncoder` is reasonably good.

Let's see if we can get an even better accuracy with `OneHotEncoder`.

Hint: `HistGradientBoostingClassifier` does not yet support sparse input data.
You might want to use `OneHotEncoder(handle_unknown="ignore",
sparse_output=False)` to force the use of a dense representation as a
workaround.

In [11]:
# Write your code here.
# Write your code here.
import time
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

# define selector
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

# define column names for each dtype
numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

# define transformers
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

# define column transformer
preprocessor = ColumnTransformer(
    [
        ("ordinal-encoder", categorical_preprocessor, categorical_columns),
        ("standard-scaler", numerical_preprocessor, numerical_columns)
    ]
)

# make pipeline
model = make_pipeline(preprocessor, HistGradientBoostingClassifier())

# running cv on entire dataset to estimate generalisability
start_time = time.time()
cv_results = cross_validate(model, data, target, cv=5)
elapsed_time = time.time() - start_time
scores = cv_results["test_score"]

print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} ± {scores.std():.3f} "
    f"with a fitting time of {elapsed_time:.3f}"
)


The mean cross-validation accuracy is: 0.873 ± 0.002 with a fitting time of 6.386


Model accuracy is identical. Computational performance worsens.

### Analysis

From an accuracy point of view, the result is almost exactly the same. The
reason is that `HistGradientBoostingClassifier` is expressive and robust
enough to deal with misleading ordering of integer coded categories (which was
not the case for linear models).

However from a computation point of view, the training time is much longer (many more features):
this is caused by the fact that `OneHotEncoder` generates more features than
`OrdinalEncoder`; for each unique categorical value a column is created.

Note that the current implementation `HistGradientBoostingClassifier` is still
incomplete, and once sparse representation are handled correctly, training
time might improve with such kinds of encodings.

The main take away message is that arbitrary integer coding of categories is
perfectly fine for `HistGradientBoostingClassifier` and yields fast training
times.

Which encoder should I use?

|                  | Meaningful order              | Non-meaningful order |
| ---------------- | ----------------------------- | -------------------- |
| Tree-based model | `OrdinalEncoder`              | `OrdinalEncoder` with reasonable depth    |
| Linear model     | `OrdinalEncoder` with caution | `OneHotEncoder`      |

<div class="admonition important alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Important</p>
<ul class="last simple">
<li><tt class="docutils literal">OneHotEncoder</tt>: always does something meaningful, but can be unnecessary
slow with trees.</li>
<li><tt class="docutils literal">OrdinalEncoder</tt>: can be detrimental for linear models unless your category
has a meaningful order and you make sure that <tt class="docutils literal">OrdinalEncoder</tt> respects this
order. Trees can deal with <tt class="docutils literal">OrdinalEncoder</tt> fine as long as they are deep
enough. However, when you allow the decision tree to grow very deep, it might
overfit on other features.</li>
</ul>
</div>

Next to one-hot-encoding and ordinal encoding categorical features,
scikit-learn offers the [`TargetEncoder`](https://scikit-learn.org/stable/modules/preprocessing.html#target-encoder).
This encoder is well suited for nominal, categorical features with high
cardinality. This encoding strategy is beyond the scope of this course,
but the interested reader is encouraged to explore this encoder.